<a href="https://colab.research.google.com/github/zimzion/EYTIP/blob/master/Functional_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np

In [0]:
def pfactorsl(x):
  if x % 2 == 0:
    yield 2
    if x // 2 > 1:
      yield from pfactorsl(x // 2)
    return

  to = int(math.sqrt(x) + .5) + 1
  for i in range(3, to, 2):
    if x % i == 0:
      yield i
      if x // i > 1:
        yield from pfactorsl(x // i)
      return
  yield x

In [48]:
n = 1023
factors = list(pfactorsl(n))
n2 = np.prod(factors)
mul_str = ' X '.join([str(e) for e in factors])
print(f'{mul_str} = {n2} = {n} ({n==n2})')

3 X 11 X 31 = 1023 = 1023 (True)
